In [1]:
#IPython magic to reload libraries
%load_ext autoreload
%autoreload 2

# Extra code so Juypter doesn't crash on mac os
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
import sys
sys.path.insert(0, 'utils')  # noqa

import numpy as np
import gym
import ntm
import interfaces

In [3]:
# Training Params
train_iter = 1000
batch_size = 1


In [4]:
# Task
env = gym.make('Copy-v0')
env = env.unwrapped
print(env.action_space)
print(env.observation_space)


Tuple(Discrete(2), Discrete(2), Discrete(5))
Discrete(6)


/Users/zachyamaoka/opt/anaconda3/envs/tf2_robo_research/lib/python3.7/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [72]:
# Controller 
Controller = ntm.NTM()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
tape_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
memory_input (InputLayer)       [(None, 10)]         0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 11)           0           tape_input[0][0]                 
                                                                 memory_input[0][0]               
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 32)           384         concatenate_1[0][0]        

In [ ]:
#Over Fit on single char
output = Controller.run(tape_input=1)
print(output)
Controller.update(output_target=4)

In [71]:
# Test RL training
output = Controller.run(tape_input=1)
Controller.update_store(1, done=True)


Objective Function J(0):  -0.14934909343719482


In [73]:
output_control_cmds = ["no","yes"]
input_control_cmds = ["left","right"]
int_to_char = ["A","B","C","D","E","F"]
render_env = True
for i in range(train_iter):
# for i in range(1):

    # Collect experince using current policy/controller
    for j in range(batch_size):
        
        episode_steps = 0
        observation = env.reset()
        if render_env: env.render()

        while True:
            episode_steps += 1
            
            #Run RL-NTM controller and get outputs
            outputs = Controller.run(tape_input=observation)

            output_content = outputs[0] # Which character to write (ignored if the above sub-action is 0) 
            output_control = outputs[1] #  Whether to write to the output tape ["no","yes"] = [0,1]
            input_control = outputs[2] # Direction to move the read head ["left","right"] = [0,1]
           
            actions = (input_control, output_control, output_content)
            
            print("Policy - obs: {} action: {}, {}, {}".format(int_to_char[observation],
                  int_to_char[output_content],
                  output_control_cmds[output_control],
                  input_control_cmds[input_control]))

            #Get output target before updating enviroment
            target = env.target[env.write_head_position]
            observation, reward, done, info = env.step(actions)
            if render_env: env.render()
                
            if output_control_cmds[output_control] == "yes":
                Controller.update_store(output_target=target, done=done)
                
            if done:
                done = False
                break
        

Total length of input instance: 4, step: 0
Observation Tape    :   DBBB  
Output Tape         :   
Targets             :   DBBB  






Policy - obs: D action: A, yes, right
Total length of input instance: 4, step: 1
Observation Tape    :   DBBB  
Output Tape         :   A
Targets             :   DBBB  

Current reward      :   -0.500
Cumulative reward   :   -0.500
Action              :   Tuple(move over input: right,
                              write to the output tape: True,
                              prediction: A)
Objective Function J(0):  -1.197824239730835
Total length of input instance: 4, step: 0
Observation Tape    :   BDBC  
Output Tape         :   
Targets             :   BDBC  






Policy - obs: B action: B, yes, right
Total length of input instance: 4, step: 1
Observation Tape    :   BDBC  
Output Tape         :   B
Targets             :   BDBC  

Current reward      :   1.000
Cumulative reward   :   1.000
Action              :   Tuple(move over input: right,
      

Total length of input instance: 2, step: 0
Observation Tape    :   BE  
Output Tape         :   
Targets             :   BE  






Policy - obs: B action: C, yes, right
Total length of input instance: 2, step: 1
Observation Tape    :   BE  
Output Tape         :   C
Targets             :   BE  

Current reward      :   -0.500
Cumulative reward   :   -0.500
Action              :   Tuple(move over input: right,
                              write to the output tape: True,
                              prediction: C)
Objective Function J(0):  -1.572768211364746
Total length of input instance: 4, step: 0
Observation Tape    :   DEAD  
Output Tape         :   
Targets             :   DEAD  






Policy - obs: D action: D, yes, right
Total length of input instance: 4, step: 1
Observation Tape    :   DEAD  
Output Tape         :   D
Targets             :   DEAD  

Current reward      :   1.000
Cumulative reward   :   1.000
Action              :   Tuple(move over input: right,
              

Policy - obs: D action: C, yes, right
Total length of input instance: 2, step: 1
Observation Tape    :   DD  
Output Tape         :   C
Targets             :   DD  

Current reward      :   -0.500
Cumulative reward   :   -0.500
Action              :   Tuple(move over input: right,
                              write to the output tape: True,
                              prediction: C)
Objective Function J(0):  -1.1067636013031006
Total length of input instance: 4, step: 0
Observation Tape    :   ECDA  
Output Tape         :   
Targets             :   ECDA  






Policy - obs: E action: B, yes, right
Total length of input instance: 4, step: 1
Observation Tape    :   ECDA  
Output Tape         :   B
Targets             :   ECDA  

Current reward      :   -0.500
Cumulative reward   :   -0.500
Action              :   Tuple(move over input: right,
                              write to the output tape: True,
                              prediction: B)
Objective Function J(0):  -1.5889542

Objective Function J(0):  -3.103529930114746
Total length of input instance: 2, step: 0
Observation Tape    :   EA  
Output Tape         :   
Targets             :   EA  






Policy - obs: E action: C, yes, right
Total length of input instance: 2, step: 1
Observation Tape    :   EA  
Output Tape         :   C
Targets             :   EA  

Current reward      :   -0.500
Cumulative reward   :   -0.500
Action              :   Tuple(move over input: right,
                              write to the output tape: True,
                              prediction: C)
Objective Function J(0):  -1.3891879320144653
Total length of input instance: 3, step: 0
Observation Tape    :   CCC  
Output Tape         :   
Targets             :   CCC  






Policy - obs: C action: D, yes, right
Total length of input instance: 3, step: 1
Observation Tape    :   CCC  
Output Tape         :   D
Targets             :   CCC  

Current reward      :   -0.500
Cumulative reward   :   -0.500
Action              :   

Total length of input instance: 3, step: 0
Observation Tape    :   DAA  
Output Tape         :   
Targets             :   DAA  






Policy - obs: D action: B, yes, right
Total length of input instance: 3, step: 1
Observation Tape    :   DAA  
Output Tape         :   B
Targets             :   DAA  

Current reward      :   -0.500
Cumulative reward   :   -0.500
Action              :   Tuple(move over input: right,
                              write to the output tape: True,
                              prediction: B)
Objective Function J(0):  -1.5616998672485352
Total length of input instance: 3, step: 0
Observation Tape    :   CDB  
Output Tape         :   
Targets             :   CDB  






Policy - obs: C action: D, yes, right
Total length of input instance: 3, step: 1
Observation Tape    :   CDB  
Output Tape         :   D
Targets             :   CDB  

Current reward      :   -0.500
Cumulative reward   :   -0.500
Action              :   Tuple(move over input: right,
           

...

Policy - obs: E action: E, yes, right
Total length of input instance: 2, step: 1
Observation Tape    :   EE  
Output Tape         :   E
Targets             :   EE  

Current reward      :   1.000
Cumulative reward   :   1.000
Action              :   Tuple(move over input: right,
                              write to the output tape: True,
                              prediction: E)
Policy - obs: E action: E, yes, right
Total length of input instance: 2, step: 2
Observation Tape    :   EE  
Output Tape         :   EE
Targets             :   EE  

Current reward      :   1.000
Cumulative reward   :   2.000
Action              :   Tuple(move over input: right,
                              write to the output tape: True,
                              prediction: E)
Objective Function J(0):  -0.06370499730110168
Total length of input instance: 3, step: 0
Observation Tape    :   EAC  
Output Tape         :   
Targets             :   EAC  






Policy - obs: E action: E, yes, right
Total 

Total length of input instance: 4, step: 0
Observation Tape    :   EACC  
Output Tape         :   
Targets             :   EACC  






Policy - obs: E action: E, yes, right
Total length of input instance: 4, step: 1
Observation Tape    :   EACC  
Output Tape         :   E
Targets             :   EACC  

Current reward      :   1.000
Cumulative reward   :   1.000
Action              :   Tuple(move over input: right,
                              write to the output tape: True,
                              prediction: E)
Policy - obs: A action: B, yes, right
Total length of input instance: 4, step: 2
Observation Tape    :   EACC  
Output Tape         :   EB
Targets             :   EACC  

Current reward      :   -0.500
Cumulative reward   :   0.500
Action              :   Tuple(move over input: right,
                              write to the output tape: True,
                              prediction: B)
Objective Function J(0):  -0.04219330474734306
Total length of input instance: 3

Policy - obs: D action: D, yes, right
Total length of input instance: 2, step: 1
Observation Tape    :   DE  
Output Tape         :   D
Targets             :   DE  

Current reward      :   1.000
Cumulative reward   :   1.000
Action              :   Tuple(move over input: right,
                              write to the output tape: True,
                              prediction: D)
Policy - obs: E action: E, yes, right
Total length of input instance: 2, step: 2
Observation Tape    :   DE  
Output Tape         :   DE
Targets             :   DE  

Current reward      :   1.000
Cumulative reward   :   2.000
Action              :   Tuple(move over input: right,
                              write to the output tape: True,
                              prediction: E)
Objective Function J(0):  -0.1681598424911499
Total length of input instance: 4, step: 0
Observation Tape    :   AABD  
Output Tape         :   
Targets             :   AABD  






Policy - obs: A action: B, yes, right
Total

Total length of input instance: 2, step: 0
Observation Tape    :   BA  
Output Tape         :   
Targets             :   BA  






Policy - obs: B action: B, yes, right
Total length of input instance: 2, step: 1
Observation Tape    :   BA  
Output Tape         :   B
Targets             :   BA  

Current reward      :   1.000
Cumulative reward   :   1.000
Action              :   Tuple(move over input: right,
                              write to the output tape: True,
                              prediction: B)
Policy - obs: A action: B, yes, right
Total length of input instance: 2, step: 2
Observation Tape    :   BA  
Output Tape         :   BB
Targets             :   BA  

Current reward      :   -0.500
Cumulative reward   :   0.500
Action              :   Tuple(move over input: right,
                              write to the output tape: True,
                              prediction: B)
Objective Function J(0):  -0.02371390163898468
Total length of input instance: 2, step: 0
Ob

Policy - obs: A action: B, yes, right
Total length of input instance: 4, step: 1
Observation Tape    :   AEEC  
Output Tape         :   B
Targets             :   AEEC  

Current reward      :   -0.500
Cumulative reward   :   -0.500
Action              :   Tuple(move over input: right,
                              write to the output tape: True,
                              prediction: B)
Objective Function J(0):  0.0
